In [1]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '20'
os.environ['MKL_NUM_THREADS'] = '20'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import datetime as dt
import copy
import statsmodels.api as sm
from matplotlib.colors import LogNorm
import xarray as xr
import pickle
import random
import time
import signal
import multiprocessing
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from scipy import interpolate
import random
import lightgbm as lgb
import itertools
from scipy import spatial
import joblib



def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import sys
sys.path.insert(1,'/home/jsperezc/jupyter/AQ_Forecast/functions/')
import postprocessing

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import RegressorChain, MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit


In [2]:
from trajectories import *

Es necesario cargar toda la información!! (una evaluación de 3 meses durante la pandemia) Con el fin de identificar limitaciones del modelo!! (y realizar una discusión entorno a esto)

In [3]:
path_files = '/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/'

files_gfs = np.concatenate([np.sort(glob.glob(path_files+'*202003*.nc')),
                np.sort(glob.glob(path_files+'*202004*.nc')),
                np.sort(glob.glob(path_files+'*202005*.nc'))])


In [4]:
#path_files = '/var/data1/AQ_Forecast_DATA/historic/GFS/validation_rev1/Vientos/'
#files_gfs = np.sort(glob.glob(path_files+'*.nc'))

lati =   6.25
loni = -75.60
delta_t = 1   # horas
leveli  = 800 # hPa  
ndays = 4

files_date_2, files_list_2, lat_dataset, lon_dataset, levels_dataset = \
         Follow(path=path_files, path_fig=None, source='GFS',
         lati=lati,loni=loni, warnings=False)

path_out   = '/var/data1/AQ_Forecast_DATA/historic/GFS/validation_rev1/Vientos/BT/'

In [5]:
def compute_save_bt(actualization_file):
    print(actualization_file)
    last_date = actualization_file.split("/")[-1][9:-3]
    date_dt = dt.datetime.strptime(last_date, "%Y%m%d%H")
    
    date_name = f'{date_dt.strftime("%Y%m%d%H")}'
    current_wind_file = f"gfs_0p25_{date_name}.nc"
    
    used_files = np.where(files_list_2 <= path_files + current_wind_file)[0]
    files_list = files_list_2[used_files]
    files_date = files_date_2[used_files]
    
    Datei = date_dt.strftime('%Y-%m-%d %H:00')
    Datef = (date_dt+dt.timedelta(hours = 115)).strftime('%Y-%m-%d %H:00') ## Este 96 es por las 96 horas del pronóstico
#        print(f'{Fechai} -----> {Fechaf}')
    name_file = f'BT_GFS.{delta_t}h.{leveli}hPa.{date_name}.{ndays}days.nc'


    # Calculamos la retrotrayectoria para un solo día
    BT = Trajectories_level_i(lati = lati, loni=loni, leveli=leveli, Fechai=Datei, Fechaf=Datef, delta_t=1,
        ndays=ndays,source='GFS',path_files=path_files,
        files_date=files_date, files_list=files_list, lat_dataset=lat_dataset, lon_dataset=lon_dataset, 
        levels_dataset=levels_dataset)
    name_file = f'BT_GFS.{delta_t}h.{leveli}hPa.{date_name}.{ndays}days.nc'
    save_nc(dictionary=BT, file_out=f'{path_out}{name_file}')
    print('finished: '+ actualization_file)

In [6]:
from multiprocessing import Pool

# processing (multiprocessing)
with Pool(20) as p:
    result = p.map(compute_save_bt, files_gfs[20:])

/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030918.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030600.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030706.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030812.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031600.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031100.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031206.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031312.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020032312.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031418.nc/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031706.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031812.nc

/var/data1/AQ_Forecast_DATA/historic

finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031000.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031006.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031712.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031718.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020032900.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020032906.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031218.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031300.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031112.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031118.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030718.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic

/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040606.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030806.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040712.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020030700.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040818.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020031912.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041000.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020032306.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041106.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020032518.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041212.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic

finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042218.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042300.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042006.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042012.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041506.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041512.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041612.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041618.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042400.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042406.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041718.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic

/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042500.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040206.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042606.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040312.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042712.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020040600.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020042818.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041206.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020043000.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020041100.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050106.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic

finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050506.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050512.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050218.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050300.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050718.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050800.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051006.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051012.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050900.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050906.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051218.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic

/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051600.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051412.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051418.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051806.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051812.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051700.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051706.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051912.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020051918.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020050418.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052000.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic

/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020053018.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020053118.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052018.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052100.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052912.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052918.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052200.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052206.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052412.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052418.nc
finished: /var/data1/AQ_Forecast_DATA/historic/GFS/historic/Vientos/gfs_0p25_2020052700.nc
/var/data1/AQ_Forecast_DATA/historic/GFS/historic